# <div align="center"> 机器分配草案 </div>

## 假设

- 同一区域的课表时间统一, 比如该区域都是每天8:10上第一节课, 8:50下课, 课间休息10分钟.

- 不存在课表调休, 比如节假日补课, 老师换课等因特殊不确定原因及不确定调课结果等

- 非上课时间任务强制终止属于正常现象

- 同一个学校, 班级的人数尽量均匀(比如: 都是35人左右)

- 用户训练任务显存占用控制在2GB以内

## 用户编号规范

```

                                           学校    用户序号(学生&教师)
                                           ---    ---
                                        3332220304001
                                        ---   ----
                                        地区   班级(教师)

```

地区码: 000 - 999

学校码: 000 - 999

班级号: 0000 - 9999  (0000:老师, 0304: 3年级4班)

用户号: 000 - 999 

## 区域

区域码|名字|调度表|机器节点
:---:|:---:|:---:|:------:
<img width=150px>|<img width=150px/>|<img width=150px/>|<img width=450px/>
100101|华东一区|周节矩阵|[192.168.1.101:8989, 192.168.1.102:8089, 192.168.1.102:8089]|
100102|华东二区|周节矩阵|[192.168.2.101:8989, 192.168.2.102:8089, 192.168.2.102:8089]|

### 调度表

调度表由指定区域里所有学校的课表汇总形成的, 按1周7天, 每天8节课(24小时), 形成一个时间矩阵,如T(1, 3)表示星期1第3节课, 而T(1, 3)的值为学校3332020302(☺), T(x, y)可以由当前的日期获取到(20200719-08:15:00 --> T(1, 1)), 而T(x, y)的值也是确定的(一个区域的排课安排是已知的).

如:

```
       3332010301: ☻ (编号为333区域201学校3年级1班: 30人)
       3332020302: ☺ (编号为333区域202学校3年级2班: 42人)
       3332030302: ◎ (编号为333区域203学校3年级2班: 29人)

              
                       星    星    星    星    星    星    星 
                       期    期    期    期    期    期    期 
                       一    二    三    四    五    六    日
                  +------------------------------------------------> 周
                  |         
                  |                     
(7:50 - 8:30)  1  |    ☻   
                  |          
               2  |                     ☻,☺            
                  |         
               3  |    ☺     ☺     ☻          ☺ 
                  |                      
               4  |    ☻          ☻,☺         ☺ 
                  |         
               5  |                ☺        ☻,☺,◎ 
                  |         
               6  |         ☺,☻    ☻          ☻
                  |         
               7  |   ☺,☻    ☻           ☻
                  |         
               8  |                
                  |         
                  |
                  v
                时间(节)

```

情景:

1. 每天的课应该不是饱和, 比如`T(1,2), T(5,7)`没有安排课程, 此时`T(1,2)=[], T(5,7)=[]`

2. 同一时刻存在多个班同时上课的情景, 比如`T(4,2)=[2010301, 2020302]` 

3. 按每天24小时算的话, 凡是不在规定课表内的时间, 统一记为`T(0, 0)`, 比如, 周末, 夜晚, 节假日, 即默认`T(x,y)=[]`



### 算力节点数(机器)

1. 按每个任务显存使用不超过2GB, 一台P40大概能运行10个任务, 则1个班40个学生(包括老师), 需要4台P40, 在此人数(40)上, 每增加1-10个人需要另增加1台P40.

2. ~~某区域下调度课表如果同一节课只存在一个班级上课, 则只需一个班级的台数(1台/10), 即为x.~~

3. ~~某区域下调度课表如果同一节课存在Y个班级同时上课, 则这个区域的机器个数按 x * max(Y)~~

4. 某区域的机器使用个数计算公式:

$$
    \bigg ( \max_{x \in X, y \in Y} \sum_{c}^{T(x, y)} S(c) + (K-1) \bigg ) / K
$$

X: 周, 取值为1, 2, 3, 4, 5, 6, 7

Y: 节, 取值为1, 2, 3, 4, 5, 6, 7, 0(表示非上课节)

S(c): 返回c班的学生人数

K: 固定常数, 如果一台P40能够运行10个显存不超过2GB的任务, 则K=10

## 调度表属性

status: 有课busy, 无课free

## 任务属性

status: running(正在运行), stopped(手动停止或者异常停止), paused(暂停), **robbed(资源被抢占)**, finished(正常结束)

## 用户属性

active: 活动, 非活动

## 场景

### 课上拒绝

```
      用户(老师&学生)                 业务服务端                         框架服务                                    
         -------                    ----------                    -----------------         
            |                           |                                 |
            |        a1                 |                                 |                               
            |-------------------------->|                                 |                               
            |        a2                 |                                 |                               
            |<--------------------------|                                 |                               
            |                           |                                 |
                                                                                                          
                                                                                                          
a1: 用户编号userid=3332030301在请求训练

a2: 后台根据当前日期时间获取T(x, y)=[3332010301, 3332020302](调度表busy), 且userid不在T(x,y)中, 拒绝 
```

### 课上清理

```

      用户(老师&学生)                  业务服务端                         框架服务                                    
         -------                    ----------                    -----------------         
            |                           |                                 |                   
            |                           |                a1               |                   
            |                           | <-----------------------------| | 
            |                           |                                 |                   
            |        a2                 |                                 |                   
            |-------------------------->|             a3                  |                   
            |                           |-------------------------------->|                   
            |                           |             a4                  |                   
            |        a5                 |<------------------------------| |                   
            |<--------------------------|                                 |                   
            |                           |                                 |                   
                                                                                              
                                                                                              
a1: 每30s上报一次内存情况                                
                                                                                              
a2: 上课时间, 老师准备上课, 登录网页, 后台根据当前日期时间获取T(x, y)=[3332010301, 3332020302]及需要的资源
                                                                                              
a3: 如果该节课资源不足, 调用框架接口清理资源, 两种方式:
    1. 传给框架需要被清理的任务列表, 框架逐个停止.
    2. 传给框架匹配模式[^3332010301*, ^3332020302*, Teac], 框架根据模式自动清理. (^表示非, *表示任意)                                                                                           
                                                                                              
a4: 框架清理后, 异步方式告知哪些用户的任务被清理掉了.                                                                                              

a5: 如果用户属性处于活动(active)状态, 需要友好提示该用户(任务被抢占)

```

### 资源满足: 课下(后台)正常调度

```
      用户(老师&学生)                  业务服务端                          框架服务
         -------                    ----------                    -----------------
            |                           |                a1               |
            |                           | <-----------------------------| |
            |                           |                                 |
            |                           |                                 |
            |                           |----+                            |
            |                           |    | a2: check T(x, y)          |
            |                           |    |                            |
            |                           |<---+                            |
            |                           |               a3: start         |
            |                           |-------------------------------->|
            |                           |                                 |
            |                           |                                 |
            |                           |                                 |
            |                           |                                 |


a1: 框架周期上报算力资源信息

a2: 定时(或者触发方式)根据当前日期时间获取T(x, y), 如果`T(x, y)=[]`, 说明调度表此刻为free状态

a3: 资源充足, 获取该区域内的所有用户任务及用户active属性, 并根据算力信息, 进行后台任务调度(active优先)
```

### 资源不足: 课下(后台)抢占调度

```
      用户(老师&学生)                 业务服务端                          框架服务
         -------                    ----------                    -----------------
            |                           |                a1               |
     active |                           | <-----------------------------| |
            |                           |                                 |
            |                           |                                 |
            |                           |----+                            |
            |                           |    | a2: check T(x,y)           |
            |                           |    |                            |
            |                           |<---+                            |
            |                           |                                 |
            |                      +----|                                 |
            |          a3: active  |    |                                 |
            |                      |    |                                 |
            |                      +--->|               a4: pause         |
            |                           | ------------------------------> |
            |                           |      xxx: paused                |
            |                           | <-----------------------------| |
            |                           |                                 |
            |                           |               a5: start         |
            |                           | ------------------------------> |
            |                           |                                 |
            |                           | <-----------------------------| |
            |                           |                                 |


a1: 框架周期上报算力资源信息

a2: 定时(或者触发方式)根据当前日期时间获取T(x, y), 如果`T(x, y)=[]`, 说明调度表此刻为free状态

a3: 资源不足, 且用户状态为active时, 查询该区域所有unactive状态的后台任务

a4: 根据资源需要情况进行puase(抢占)掉unactive用户状态的后台任务, 为active用户任务准备资源

a5: 资源满足, 启动active用户的任务

```